## Messung der OCR-Qualität in Python

In diesem Notebook bewerten wir die OCR-Qualität. Wir werden denselben Beispielartikel zur Bewertung verwenden:
<!-- In this notebook we evaluate OCR quality. We'll use the same sample article for evaluation: -->


![grippe.png](grippe.png)

### Durchführen von OCR

Zuerst führen wir die OCR durch: <!-- Let us OCR it first: -->

In [ ]:
import pytesseract
from PIL import Image

In [ ]:
## for interactivity in jupyter books
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), 
                                         lang='frk') # Wir verwenden das Fraktur-OCR-Modell

In [ ]:
print(ocr_output)

### Manuell die 'Ground Truth' zur Bewertung erstellen <!-- ### Manually create  the 'ground truth' to evaluate against -->

In [ ]:
output_notebook()

capture_ground_truth = TextInput(value="""Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.""", 
                                 title="Please insert corrected string: ") 

ground_truth = capture_ground_truth.value.strip()

# Layout and display
layout = column(capture_ground_truth)

show(layout)

#ground_truth = input('Please insert corrected string: ')

In [ ]:
print(ground_truth)

### Berechnung von Precision, Recall und F1-Measure <!-- ### Calculate precision, recall, and F1-measure -->

In [ ]:
from aux.measure_ocr_quality import measure_ocr_quality

In [ ]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')